In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [128]:
url_inicial = 'https://www.probdes.iiec.unam.mx/index.php/pde/issue/archive/9'
url_root = 'https://www.probdes.iiec.unam.mx/index.php/pde/issue/archive/9'
r=requests.get(url_inicial)

In [129]:
soup = BeautifulSoup(r.text, 'html.parser')

In [130]:
box = soup.find('div', class_='issues media-list')
volumen=box.findAll('div', class_='media-body')
vol = [x.find('a').get('href')for x in volumen]
vol=[urljoin(url_root,i) for i in vol]
vol

['https://www.probdes.iiec.unam.mx/index.php/pde/issue/view/3617',
 'https://www.probdes.iiec.unam.mx/index.php/pde/issue/view/3613',
 'https://www.probdes.iiec.unam.mx/index.php/pde/issue/view/3592',
 'https://www.probdes.iiec.unam.mx/index.php/pde/issue/view/3591']

In [131]:
vol2 =[]
for i in vol: 
    url_inicial1=i 
    r1 = requests.get(url_inicial1)
    soup1 = BeautifulSoup(r1.text, 'html.parser')
    box1 = soup1.find('div', class_='media-list') 
    volumen1=box1.findAll('div', class_='article-summary media')
    vol1 = [x.find('a').get('href')for x in volumen1]
    vol2+=vol1

In [132]:
vol2

['https://www.probdes.iiec.unam.mx/index.php/pde/article/view/45049',
 'https://www.probdes.iiec.unam.mx/index.php/pde/article/view/44977',
 'https://www.probdes.iiec.unam.mx/index.php/pde/article/view/44561',
 'https://www.probdes.iiec.unam.mx/index.php/pde/article/view/44524']

In [133]:
def get_url_items(sopa,url):
    box1 = soup1.find('div', class_='media-list') 
    volumen1=box1.findAll('div', class_='article-summary media')
    vol1 = [x.find('a').get('href')for x in volumen1]
    return vol2

In [134]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links) 
    next_a=s_p.select('li.next > a')
    if not next_a or not next_a[0].get('href'):
        break

Estoy en la pagina https://www.probdes.iiec.unam.mx/index.php/pde/issue/archive/9


In [135]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

4

In [136]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [137]:
#funcion para iniciar el escraper de cada libro
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tire='Problemas del Desarrollo. Revista Latinoamericana de Economía'
    a='V. Ciencias Sociales'
    tem='Economía, Desarrollo Económico'
    s_item=BeautifulSoup(r.text,'html.parser')
 #titulo de revista
    try:
        titulo=tire
        content_book['Titulo Revista']=tire
    except AttributeError:
        content_book['Titulo Revista']=None
    #area
    try:
        area=a
        content_book['Area']=a
    except AttributeError:
        content_book['Area']=None
    #tematica
    try:
        tema=tem
        content_book['Tematica']=tem
    except AttributeError:
        content_book['Tematica']=None
    #titulo articulo
    try:
        titu=s_item.find('h1', class_='page-header').get_text(strip=True)
        content_book['Titulo Articulo']=titu
    except AttributeError:
        content_book['Titulo Articulo']=None
    #resumen
    try:
        resu=s_item.find('div', class_='article-abstract').get_text(strip=True)
        content_book['Resumen']=resu#.replace("Resumen","")
    except AttributeError:
        content_book['Resumen']=None
    #abstract
    try:
        abst=s_item.find('div', id='item cover_image').get_text(strip=True)
        content_book['Abstract']=abst
    except AttributeError:
        content_book['Abstract']=None
    #Link incial
    try:
        linkart=url
        content_book['Link Articulo']=linkart
    except AttributeError:
        content_book['Link Articulo']=None
    #link articulo
    try:
        link=s_item.find('a', class_='galley-link btn btn-primary pdf').get('href')
        content_book['Link PDF']=urljoin(url_root, link)
    except AttributeError:
        content_book['Link PDF']=None
    return content_book

In [138]:
list_scraper=list_scraper[0:4]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

estas escrapeando la pag 0
estas escrapeando la pag 1
estas escrapeando la pag 2
estas escrapeando la pag 3


In [139]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

,Titulo Revista,Area,Tematica,Titulo Articulo,Resumen,Abstract,Link Articulo,Link PDF
0,Problemas del Desarrollo. Revista Latinoameric...,V. Ciencias Sociales,"Economía, Desarrollo Económico",A nuestros lectores,None,None,https://www.probdes.iiec.unam.mx/index.php/pde...,https://www.probdes.iiec.unam.mx/index.php/pde...
1,Problemas del Desarrollo. Revista Latinoameric...,V. Ciencias Sociales,"Economía, Desarrollo Económico",A nuestros lectores,None,None,https://www.probdes.iiec.unam.mx/index.php/pde...,https://www.probdes.iiec.unam.mx/index.php/pde...
2,Problemas del Desarrollo. Revista Latinoameric...,V. Ciencias Sociales,"Economía, Desarrollo Económico",A nuestros lectores,None,None,https://www.probdes.iiec.unam.mx/index.php/pde...,https://www.probdes.iiec.unam.mx/index.php/pde...
3,Problemas del Desarrollo. Revista Latinoameric...,V. Ciencias Sociales,"Economía, Desarrollo Económico",A nuestros lectores,None,None,https://www.probdes.iiec.unam.mx/index.php/pde...,https://www.probdes.iiec.unam.mx/index.php/pde...


In [140]:
df_catalogo.to_csv('Revista69.09.csv', index=False)